In [46]:
import pandas as pd

In [47]:
form = "1"
version = "V4"
rev = "_rev1"
year = "2024"

company_sym = "BAM"
# company_sym = "BAY"
# company_sym = "CIMBT"
# company_sym = "LHFG"
# company_sym = "SCB"

# company_sym = "ACAP"
# company_sym = "AEONTS"
# company_sym = "JMT"
# company_sym = "SAWAD"

# company_sym = "BYD"
# company_sym = "KGI"
# company_sym = "MFC"

# company_sym = "BUI"
# company_sym = "NKI"
# company_sym = "TIPH"
# company_sym = "TSI"

# company_sym = "3BBIF"
# company_sym = "BKKCP"
# company_sym = "BTSGIF"
# company_sym = "FTREIT"
# company_sym = "MJLF"

prod_result_path = f"./Listed_Mapping_Prod/{year}/{company_sym}_Q2_{year}.csv"
ai_result_path = f"./AI_Mapping_Result/{version}/{form}_{company_sym}_mapping_result_{version}{rev}.csv"

output_csv_path = f"./Compare_Result/{form}_{company_sym}_{version}{rev}_compare.csv"
output_txt_path = f"./Compare_Result/{form}_{company_sym}_{version}{rev}_compare.txt"

print(prod_result_path)
print(ai_result_path)
print(output_csv_path)
print(output_txt_path)

./Listed_Mapping_Prod/2024/BAM_Q2_2024.csv
./AI_Mapping_Result/V4/1_BAM_mapping_result_V4_rev1.csv
./Compare_Result/1_BAM_V4_rev1_compare.csv
./Compare_Result/1_BAM_V4_rev1_compare.txt


In [48]:
th_month = [
    'มกราคม', 
    'กุมภาพันธ์', 
    'มีนาคม',
    'เมษายน',
    'พฤษภาคม',
    'มิถุนายน',
    'กรกฎาคม',
    'สิงหาคม',
    'กันยายน',
    'ตุลาคม',
    'พฤศจิกายน',
    'ธันวาคม',
]

th_month_short = [
    'ม.ค.', 
    'ก.พ.', 
    'มี.ค.',
    'เม.ย.',
    'พ.ค.',
    'มิ.ย.',
    'ก.ค.',
    'ส.ค.',
    'ก.ย.',
    'ต.ค.',
    'พ.ย.',
    'ธ.ค.',
]

In [49]:
def replace_comma(name):
    return name.replace(",", "")

def replace_month(name):
    for month in th_month:
        if (month in name):
            name = name.replace(month, "")

    for month in th_month_short:
        if (month in name):
            name = name.replace(month, "")
    return name

In [50]:
dtype = {
    'query': 'string',
    'set_code': 'string'
}

ai_df = pd.read_csv(ai_result_path, encoding='utf-8', dtype=dtype)

ai_df_query = ai_df['query']
ai_df_query.replace(to_replace=' ', value='', regex=True, inplace=True)
ai_df_query.replace(to_replace=r'[0-9]', value='', regex=True, inplace=True)
ai_df['query'] = ai_df_query

ai_df_query = ai_df['query']
for index, row in ai_df_query.items():
    if (pd.isna(row)):
        continue
    name = row
    name_replace = replace_month(name)
    name_replace = replace_comma(name_replace)
    ai_df['query'].iloc[index] = name_replace

ai_df.iloc[1]['query']

'รายการระหว่างธนาคารและตลาดเงินสุทธิ-เงินฝากสถาบันการเงิน'

In [51]:
dtype = {
    'account_name': 'string',
    'account_code': 'string',
    'parent_account_code': 'string',
}

prod_df = pd.read_csv(prod_result_path, encoding='utf-8', dtype=dtype)
prod_acc_name = prod_df['account_name']
prod_acc_name.replace(to_replace=r'\{{\w+\}}', value='', regex=True, inplace=True)
prod_df['account_name'] = prod_acc_name
prod_df.iloc[1]['account_name']

'รายการระหว่างธนาคารและตลาดเงินสุทธิ-เงินฝากสถาบันการเงิน'

In [60]:
compared_list = []

for index, row in ai_df.iterrows():
    ai_query = row['query']
    ai_acc_code = row['set_code']
    ai_acc_name = row['item_in_taxonomy']
    ai_fs_type = row['table_name']
    ai_reason = row['reasoning']

    prod_acc_code = ""
    prod_acc_name = ""
    prod_parent_acc_code = ""
    unmatch_remark = ""

    if (not pd.isna(ai_acc_code)):
        select_index = []
        for i, r in prod_df.iterrows():
            prod_acc_name = r['account_name']
            if (prod_acc_name in ai_query or ai_query in prod_acc_name):
                select_index.append(i)

        prod_selected_df = prod_df.iloc[select_index,:]
    
        is_match = False
        is_match_parent = False  
        
        for i, r in prod_selected_df.iterrows():
            current_acc_name = r['account_name']
            current_acc_code = r['account_code']
            current_parent_acc_code = r['parent_account_code']
            
            if (current_acc_code == ai_acc_code):
                prod_acc_code = current_acc_code
                prod_acc_name = current_acc_name
                is_match = True
            elif (not pd.isna(current_parent_acc_code) and current_parent_acc_code == ai_acc_code):
                prod_parent_acc_code = current_acc_code
                prod_acc_name = current_acc_name
                is_match_parent = True
    
        if (is_match == False and is_match_parent == False):
            print(select_index)
            print(f"Not found {ai_acc_code}-{ai_query}-{ai_acc_name} in PROD mapping\n\n")
    else:
        print(f"Null {ai_acc_code}-{ai_query}-{ai_acc_name}\n\n")
        unmatch_remark = ai_reason

    compare_item = {
        "taxonomy_account_name": ai_acc_name,
        "ai_account_code": ai_acc_code,
        "ai_acc_name": ai_acc_name,
        "prod_account_name": prod_acc_name,
        "prod_account_code": prod_acc_code,
        "prod_parent_account_code": prod_parent_acc_code,
        "is_match": str(ai_acc_code) == str(prod_acc_code),
        "is_match_parent": str(ai_acc_code) == str(prod_parent_acc_code),
        "unmatch_remark": ""
    }
    compared_list.append(compare_item)

[3, 100]
Not found 102630-เงินลงทุนในการร่วมค้า-เงินลงทุนในการร่วมค้า in PROD mapping


[5, 92]
Not found 102050-ลูกหนี้ขายผ่อนชำระและดอกเบี้ยค้างรับสุทธิ-เงินให้สินเชื่อแก่ลูกหนี้และดอกเบี้ยค้างรับ - สุทธิ เงินให้สินเชื่อแก่ลูกหนี้ in PROD mapping


[12, 95]
Not found 104900-เงินทดรองจ่ายค่าธรรมเนียมในการซื้อทรัพย์และอื่นๆสุทธิ-เงินรอรับจากการขายทอดตลาด in PROD mapping


[17]
Not found 204010-ดอกเบี้ยค้างจ่าย-เจ้าหนี้ภาษีค้างจ่าย in PROD mapping


[19]
Not found 204010-ภาษีค้างจ่าย-เจ้าหนี้ที่เกิดจากภาษีอื่นหรือรายการอื่นภายใต้กฎหมาย เจ้าหนี้ภาษีค้างจ่าย in PROD mapping


[20, 79]
Not found 204010-ค่าใช้จ่ายค้างจ่าย-เจ้าหนี้ภาษีค้างจ่าย in PROD mapping


[21]
Not found 200200-เจ้าหนี้สถาบันการเงินจากการซื้อลูกหนี้-รายการระหว่างธนาคารและตลาดเงิน in PROD mapping


[27]
Not found 319914-องค์ประกอบอื่นของส่วนของเจ้าของ-ส่วนเกินทุนจากการเปลี่ยนแปลงมูลค่าเงินลงทุน-ส่วนเกิน (ต่ำกว่า) ทุน ส่วนเกิน (ต่ำกว่า) ทุนจากการวัดมูลค่าเงินลงทุนในตราสารทุนด้วยมูลค่ายุติธรรมผ่านกำไรขาดทุนเบ็ดเสร็จอื่น in

---

In [ ]:
count_unmatch = 0
count_match_parent = 0

unmatch_list = []
for item in compared_list:
    if (item['is_match'] == False):
        if (item['is_match_parent'] == True):
            count_match_parent += 1
            print("Match Parent")
        if (item['ai_account_code'] == "" and item['ai_acc_name'] == ""):
            print("Not Match accode or name with AI")
        count_unmatch += 1
        print(item)
        unmatch_list.append(item)
        print("\n")

print(f"Total Unmatch: {count_unmatch}")
print(f"Total Match Parent: {count_match_parent}")

In [ ]:
count_no_code = 0
for item in compared_list:
    if (item['is_match'] == False):
        if (item['ai_account_code'] == ''):
            count_no_code += 1
            print(item['prod_account_code'])

print(f"No Code From AI: {count_no_code}")

In [ ]:
unmatch_df = pd.DataFrame(unmatch_list)
unmatch_df.head()

In [ ]:
unmatch_df.to_csv(output_csv_path, encoding='utf-8', index=False, header=True)

### Calculate Accuracy

In [ ]:
total_prod_taxonomy = len(prod_df.index)

In [ ]:
correct_map = total_prod_taxonomy - count_unmatch
accuracy = (correct_map / total_prod_taxonomy) * 100
print(f"Accuracy: {accuracy} %")

In [ ]:
partial_correct_map = total_prod_taxonomy - count_no_code
accuracy_parent_map = (partial_correct_map / total_prod_taxonomy) * 100
print(f"Accuracy (include parent map): {accuracy_parent_map} %")

In [ ]:
print(prod_result_path)
print(ai_result_path)

with open(output_txt_path, 'w') as txt_file:
    txt_file.write(f"Reading AI Result from: {ai_result_path}\n")
    txt_file.write(f"Reading Prod Result from: {prod_result_path}\n\n")
    
    txt_file.write(f"Total Taxonomy item from Prod: {total_prod_taxonomy}\n")
    txt_file.write(f"Not Match between Prod and AI: {count_unmatch}\n")
    txt_file.write(f"Match between Parent Prod and AI: {count_match_parent}\n\n")
    
    txt_file.write(f"Accuracy: {accuracy} % \n")
    txt_file.write(f"Accuracy (include parent map): {accuracy_parent_map} % \n\n")
    
    txt_file.write(f"Detail of unmatch item: {output_csv_path} % \n\n")